<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e55beef0255375a93d0525c5f6549665ec15ae7141c727a93e1b9638c3d950c3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-25 11:46:13
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.39 C
-------------------
Today PnL: -44.96 K (-0.32%)
Current PnL: -20.38 L (-13.71%)
CY Booked + Current PnL: -8.94 L (-6.02%)
-------------------
Total profit:  1.36 L
Total loss:  -21.74 L
-------------------
Total Booked + Current PnL: 18.04 L (14.82%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.22%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 86.25 L (61.96%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.19%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.85,58.0,H-MC,3.85,194900.0,30230.0,13097.0,0.25,18.36,6.72,26.31,89.0,2.31,1.40,27.81,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.87,43.0,M-SC,2.89,85268.0,-15509.0,15604.0,0.54,-15.39,18.30,0.09,245.0,-0.99,0.61,12.02,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-45.32,57.0,H-MC,10.48,109896.0,9392.0,19715.0,0.61,9.35,17.94,28.96,79.0,0.48,0.79,46.49,MH,ATH,METALS
84,VOLTAS,1530.00,-1.92,46.0,H-MC,3.05,205665.0,13923.0,23837.0,-0.09,7.26,11.59,19.69,99.0,0.58,1.48,14.55,XY25,NTT,AC
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.29,214734.0,4368.0,24415.0,-0.47,2.08,11.37,13.68,37.0,0.18,1.54,18.94,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,82920.0,-19290.0,112680.0,2.14,-18.87,135.89,91.37,208.0,-0.17,0.60,34.61,XY25,NTT,FINANCE
46,KPIGREEN,731.05,0.19,40.0,H-SC,9.19,119561.0,-5736.0,64659.0,1.99,-4.58,54.08,47.03,141.0,-0.09,0.86,49.70,MH,ATH,POWER
66,SIS,528.00,2088.08,41.0,H-SC,4.45,88294.0,-22738.0,46346.0,1.33,-20.48,52.49,21.26,156.0,-0.49,0.63,19.29,OX40N,NTT,MISC
8,AWL,485.00,-65.61,47.0,H-MC,3.30,244480.0,-57271.0,218687.0,1.27,-18.98,89.45,53.50,116.0,-0.26,1.76,8.59,XY24,NTT,FMCG
32,HINDZINC,730.22,22.12,58.0,M-LC,2.41,198991.0,-6085.0,118658.0,1.00,-2.97,59.63,54.89,52.0,-0.05,1.43,20.41,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.00,-24.14,36.0,M-LC,2.27,240143.0,-36380.0,146463.0,-3.13,-13.16,60.99,39.81,54.0,-0.25,1.73,15.09,XY24,NTT,AUTO
71,TANLA,1963.11,-17.04,63.0,H-SC,16.32,222416.0,-48345.0,378285.0,-2.94,-17.86,170.08,121.86,133.0,-0.13,1.60,73.99,AR,ATH,IT
26,GREENPANEL,537.00,205.56,45.0,M-SC,0.86,136420.0,-46658.0,125629.0,-2.19,-25.49,92.09,43.14,230.0,-0.37,0.98,28.32,XY24,NTT,MISC
55,RAJOOENG,143.10,-41.44,26.0,H-SC,13.47,88800.0,-13700.0,54301.0,-2.15,-13.37,61.15,39.61,114.0,-0.25,0.64,0.00,AR,ATH,MISC
56,RELAXO,1176.00,-40.36,33.0,H-SC,2.26,81329.0,-63831.0,139756.0,-2.09,-43.97,171.84,52.31,136.0,-0.46,0.58,8.86,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.45,42.0,H-MC,2.12,135069.0,-719.0,43465.0,0.17,-0.53,32.18,31.48,107.0,-0.02,0.97,11.52,X40,ATH,INSURANCE
23,DABUR,735.0,-7.62,39.0,H-MC,6.85,249896.0,1828.0,106581.0,-0.49,0.74,42.65,43.70,102.0,0.02,1.80,15.29,XY24,BTT,FMCG
86,WIPRO,420.0,-14.25,39.0,M-LC,5.49,150666.0,-279.0,109730.0,-0.65,-0.18,72.83,72.51,53.0,-0.00,1.08,5.83,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,49.0,H-SC,9.91,121284.0,-21195.0,40715.0,-0.20,-14.88,33.57,13.70,163.0,-0.52,0.87,43.15,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.87,43.0,M-SC,2.89,85268.0,-15509.0,15604.0,0.54,-15.39,18.30,0.09,245.0,-0.99,0.61,12.02,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.27,220815.0,-48852.0,85190.0,0.03,-18.12,38.58,13.47,138.0,-0.57,1.59,11.69,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,35.0,H-SC,1.98,141243.0,-34660.0,76681.0,0.30,-19.70,54.29,23.89,149.0,-0.45,1.01,21.98,OX40N,NTT,CERAMICS
66,SIS,528.0,2088.08,41.0,H-SC,4.45,88294.0,-22738.0,46346.0,1.33,-20.48,52.49,21.26,156.0,-0.49,0.63,19.29,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,69.0,M-MC,10.56,236723.0,11761.0,155882.0,0.83,5.23,65.85,74.53,192.0,0.08,1.70,39.17,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.85,58.0,H-MC,3.85,194900.0,30230.0,13097.0,0.25,18.36,6.72,26.31,89.0,2.31,1.40,27.81,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,46.0,H-MC,3.05,205665.0,13923.0,23837.0,-0.09,7.26,11.59,19.69,99.0,0.58,1.48,14.55,XY25,NTT,AC
17,CAMS,5211.76,-3.83,50.0,H-SC,1.82,109388.0,7384.0,36547.0,0.61,7.24,33.41,43.06,122.0,0.20,0.79,25.59,X40N,ATH,MISC
23,DABUR,735.00,-7.62,39.0,H-MC,6.85,249896.0,1828.0,106581.0,-0.49,0.74,42.65,43.70,102.0,0.02,1.80,15.29,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,57.0,H-MC,10.48,109896.0,9392.0,19715.0,0.61,9.35,17.94,28.96,79.0,0.48,0.79,46.49,MH,ATH,METALS
11,BANKINDIA,190.00,-27.95,63.0,H-MC,7.88,185021.0,5213.0,107201.0,-0.57,2.90,57.94,62.52,88.0,0.05,1.33,37.97,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-43.69,35.0,M-SC,1.54,95265.0,3766.0,74983.0,-1.99,4.12,78.71,86.07,223.0,0.05,0.68,38.34,XR,NTT,DURABLES
37,INDIAMART,4850.92,-55.01,26.0,H-SC,10.46,125854.0,2518.0,131241.0,-0.09,2.04,104.28,108.45,119.0,0.02,0.90,24.85,AR,ATH,MISC
86,WIPRO,420.00,-14.25,39.0,M-LC,5.49,150666.0,-279.0,109730.0,-0.65,-0.18,72.83,72.51,53.0,-0.00,1.08,5.83,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.74,33.0,H-LC,7.98,288950.0,-26892.0,139563.0,0.07,-8.51,48.30,35.67,5.0,-0.19,2.08,4.16,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-15.81,35.0,H-LC,7.40,212370.0,-39398.0,92615.0,-1.79,-15.65,43.61,21.13,27.0,-0.43,1.53,14.23,X40,ATH,PAINTS
74,TCS,4476.75,-29.33,36.0,H-LC,12.08,282197.0,-63759.0,138615.0,-1.10,-18.43,49.12,21.64,1.0,-0.46,2.03,0.00,X40,ATH,IT
44,JIOFIN,387.00,-2.93,37.0,H-LC,13.15,263762.0,-3603.0,69053.0,-0.02,-1.35,26.18,24.48,48.0,-0.05,1.90,52.88,XY24,BTT,FINANCE
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195673.0,-4465.0,25359.0,-0.21,-2.23,12.96,10.44,4.0,-0.18,1.41,3.21,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,47.0,H-LC,1.67,234855.0,-11296.0,90513.0,0.23,-4.59,38.54,32.18,16.0,-0.12,1.69,29.80,X200,ATH,IT
42,ITC,452.0,-39.61,37.0,H-LC,2.09,195673.0,-4465.0,25359.0,-0.21,-2.23,12.96,10.44,4.0,-0.18,1.41,3.21,X40,NTT,FMCG
65,SIEMENS,4671.5,-0.47,49.0,H-LC,2.81,159790.0,-26305.0,73791.0,-0.16,-14.14,46.18,25.51,15.0,-0.36,1.15,17.75,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,43.0,H-LC,4.29,214734.0,4368.0,24415.0,-0.47,2.08,11.37,13.68,37.0,0.18,1.54,18.94,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.64,46.0,H-LC,4.67,274927.0,9501.0,47287.0,-0.31,3.58,17.20,21.40,11.0,0.20,1.98,11.64,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.33,36.0,H-LC,12.08,282197.0,-63759.0,138615.0,-1.10,-18.43,49.12,21.64,1.0,-0.46,2.03,0.00,X40,ATH,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195673.0,-4465.0,25359.0,-0.21,-2.23,12.96,10.44,4.0,-0.18,1.41,3.21,X40,NTT,FMCG
83,VBL,671.64,-19.74,33.0,H-LC,7.98,288950.0,-26892.0,139563.0,0.07,-8.51,48.30,35.67,5.0,-0.19,2.08,4.16,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.94,40.0,H-LC,9.87,223709.0,-18215.0,81475.0,-1.21,-7.53,36.42,26.15,8.0,-0.22,1.61,5.71,X40,ATH,IT
80,UNITDSPR,1644.00,-11.66,57.0,H-LC,7.10,231495.0,-2649.0,51276.0,-0.37,-1.13,22.15,20.77,86.0,-0.05,1.66,5.71,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.47,49.0,H-LC,2.81,159790.0,-26305.0,73791.0,-0.16,-14.14,46.18,25.51,15.0,-0.36,1.15,17.75,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195673.0,-4465.0,25359.0,-0.21,-2.23,12.96,10.44,4.0,-0.18,1.41,3.21,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,43.0,H-LC,4.93,209596.0,5096.0,36323.0,-0.53,2.49,17.33,20.25,10.0,0.14,1.51,11.44,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-15.81,35.0,H-LC,7.40,212370.0,-39398.0,92615.0,-1.79,-15.65,43.61,21.13,27.0,-0.43,1.53,14.23,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.29,214734.0,4368.0,24415.0,-0.47,2.08,11.37,13.68,37.0,0.18,1.54,18.94,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.33,36.0,H-LC,12.08,282197.0,-63759.0,138615.0,-1.10,-18.43,49.12,21.64,1.0,-0.46,2.03,0.00,X40,ATH,IT
40,INFY,2275.00,-18.16,48.0,H-LC,8.07,318350.0,5462.0,166211.0,0.00,1.75,52.21,54.87,3.0,0.03,2.29,8.51,X40,BTT,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195673.0,-4465.0,25359.0,-0.21,-2.23,12.96,10.44,4.0,-0.18,1.41,3.21,X40,NTT,FMCG
83,VBL,671.64,-19.74,33.0,H-LC,7.98,288950.0,-26892.0,139563.0,0.07,-8.51,48.30,35.67,5.0,-0.19,2.08,4.16,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,47.0,H-LC,8.28,250910.0,-10709.0,129921.0,-0.19,-4.09,51.78,45.57,7.0,-0.08,1.80,6.76,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6750.00,50.0,L-SC,5.38,75150.0,-18660.0,95275.0,-1.13,-19.89,126.78,81.67,269.0,-0.20,0.54,45.53,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,50.0,L-SC,42.97,84154.0,-29395.0,69444.0,-1.75,-25.89,82.52,35.27,268.0,-0.42,0.60,105.97,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,82920.0,-19290.0,112680.0,2.14,-18.87,135.89,91.37,208.0,-0.17,0.60,34.61,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-6.52,41.0,H-SC,11.45,86963.0,-14000.0,108530.0,-0.06,-13.87,124.80,93.63,148.0,-0.13,0.62,29.53,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,50.0,H-SC,12.69,94155.0,-3825.0,25431.0,-0.73,-3.90,27.01,22.05,152.0,-0.15,0.68,36.86,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1198.25,69.0,M-SC,11.44,176639.0,-4504.0,49971.0,-1.42,-2.49,28.29,25.10,235.0,-0.09,1.27,41.56,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-17.04,63.0,H-SC,16.32,222416.0,-48345.0,378285.0,-2.94,-17.86,170.08,121.86,133.0,-0.13,1.60,73.99,AR,ATH,IT
61,SAMMAANCAP,326.00,-171.59,58.0,M-SC,5.51,82920.0,-19290.0,112680.0,2.14,-18.87,135.89,91.37,208.0,-0.17,0.60,34.61,XY25,NTT,FINANCE
60,SAIL,228.00,49.96,69.0,M-MC,10.56,236723.0,11761.0,155882.0,0.83,5.23,65.85,74.53,192.0,0.08,1.70,39.17,XY24,BTT,STEEL
7,ATULAUTO,844.00,3678.38,52.0,M-SC,6.58,117705.0,-29663.0,81475.0,-1.76,-20.13,69.22,35.16,236.0,-0.36,0.85,20.22,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.26
1,25,43.74
2,50,74.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.93
LC    32.44
MC    23.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.25
X40      14.78
X40N     11.41
XR       11.25
XY25     10.14
AR        8.35
OX40N     7.85
X200      1.69
MH        1.65
X5K       1.43
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.40
H-LC    26.01
H-MC    20.69
M-SC    13.03
M-LC     5.38
M-MC     2.67
L-SC     1.50
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.00,-5.46,40.27
IT,12.97,-16.62,80.21
FINANCE,10.07,-17.09,68.18
MISC,6.98,-18.17,83.87
BANKS,6.16,-16.46,77.73
PAINTS,5.65,-17.65,35.20
ELECTRICAL,5.48,-7.88,45.73
INSURANCE,3.76,-5.74,45.27
AC,3.58,2.71,13.58


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3172618.0,22
XR,1345066.0,14
AR,1170715.0,9
X40,793438.0,10
X40N,616907.0,9
OX40N,552889.0,10
XY25,429865.0,7
SR,249918.0,2
X5K,118658.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3702097.0,29
M-SC,1433587.0,17
H-MC,1227539.0,15
H-LC,1203554.0,15
M-LC,402171.0,4
M-MC,305528.0,2
L-SC,251359.0,3
L-MC,60529.0,1
L-LC,38597.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237221.0      6
M-SC       XY24       820955.0      7
H-SC       AR         798192.0      5
           XR         781566.0      7
H-MC       XY24       566078.0      4
H-LC       X40        504275.0      5
H-SC       X40N       310166.0      4
M-MC       XY24       305528.0      2
H-SC       OX40N      260375.0      4
           SR         249918.0      2
H-LC       X40N       227162.0      3
H-MC       X40        214957.0      4
H-LC       AR         203712.0      2
M-LC       XY24       173783.0      2
H-MC       XY25       169749.0      2
M-SC       AR         168811.0      2
L-SC       XR         164719.0      2
M-SC       OX40N      135614.0      4
           XR         121321.0      2
M-LC       X5K        118658.0      1
M-SC       XY25       112680.0      1
M-LC       XR         109730.0      1
H-LC       XY25       108839.0      3
H-MC       XR         107201.0      1
H-LC       X200        90513.0      1
L-SC       OX40N       86640.0      1
H-MC       X40N        79579.0      2
M-SC       X40         74206.0      1
H-MC       OX40N       70260.0      1
H-LC       XY24        69053.0      1
H-SC       MH          64659.0      1
L-MC       XR          60529.0      1
L-LC       XY25        38597.0      1
H-MC       MH          19715.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
